In [29]:
from __future__ import division, print_function, absolute_import
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
import numpy
import PIL
from PIL import Image
np.random.seed(1337)  # for reproducibility

import random
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten
from keras.optimizers import RMSprop
from keras import backend as K
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [46]:
learning_rate = 0.01
num_steps = 20
batch_size = 20
total_numbers = 291
display_step = 1000
examples_to_show = 10

# Network Parameters
num_hidden_1 = 64 # 1st layer num features
num_hidden_2 = 16 # 2nd layer num features (the latent dim)
num_input = 21
num_classes = 5

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])


weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
    'classifier_h': tf.Variable(tf.random_normal([num_hidden_2, num_classes])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([num_input])),
    'classifier_b': tf.Variable(tf.random_normal([num_classes])),
}

In [47]:
# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    return layer_2


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op

label_pred = tf.nn.softmax(tf.add(tf.matmul(encoder_op, weights['classifier_h']), biases['classifier_b']))
y_clipped = tf.clip_by_value(label_pred, 1e-10, 0.9999999)


# Targets (Labels) are the input data.
y_true = X
label_true = Y

# Define loss and optimizer, minimize the squared error
loss_autoencoder = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
cross_entropy_loss = -tf.reduce_mean(tf.reduce_sum(label_true * tf.log(y_clipped)
                         + (1 - label_true) * tf.log(1 - y_clipped), axis=1))
loss_total = loss_autoencoder+cross_entropy_loss

optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss_total)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [4]:
from numpy import genfromtxt
my_data = genfromtxt('/home/aniruddha/Documents/deep_learning_papers/DataBiplab/Cgamma_VHc.csv', delimiter=',')
my_data1 = genfromtxt('/home/aniruddha/Documents/deep_learning_papers/DataBiplab/Cgamma_VVc.csv', delimiter=',')

In [14]:
data = my_data[1:,24:]
data1 = my_data1[1:,24:]
print(data.shape)
print(data.shape)

(291, 21)
(291, 21)


In [6]:
print(data.max())

0.142512947320938


In [15]:
data = np.array(data)
data1 = np.array(data1)
print(data.shape)
print(data1.shape)

(291, 21)
(291, 21)


In [8]:
import pandas
colnames = ['Latitude', 'Longitude', 'CType', 'Remarks', 'Index']
data1 = pandas.read_csv('/home/aniruddha/Documents/deep_learning_papers/DataBiplab/CropClean.csv', names=colnames)

In [9]:
ctype = data1.CType.tolist()
print(ctype)

['CType', 'PD', 'PD', 'PD', 'SC', 'JW', 'SC', 'SC', 'SC', 'SC', 'JW', 'SC', 'SC', 'SC', 'PP', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'BA', 'BA', 'SC', 'SC', 'SC', 'JW', 'PD', 'SC', 'BA', 'SC', 'BA', 'PD', 'PD', 'SC', 'PP', 'PD', 'PD', 'PD', 'PD', 'PD', 'BA', 'SC', 'SC', 'BA', 'BA', 'JW', 'JW', 'SC', 'PD', 'BA', 'JW', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'PD', 'BA', 'SC', 'PD', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'JW', 'PD', 'PD', 'SC', 'JW', 'PD', 'PD', 'BA', 'BA', 'SC', 'SC', 'SC', 'BA', 'PP', 'PP', 'BA', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'PP', 'SC', 'SC', 'SC', 'SC', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'SC', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'JW', 'SC', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'BA', 'BA', 'PD', 'PD', 'SC', 'PD', 'SC', 'PD', 'BA', 'JW', 'PD', 'SC', 'JW', 'SC', 'SC', 'JW', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'PD', 'SC', 'SC', 'PD', 'SC', 'JW', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'JW', 'PD', 'SC', 'SC', 'JW', 

In [10]:
p = np.array(ctype)
print(p.shape)

(292,)


In [11]:
lbl1 = ctype
label = []
for i in range(1,292):
    if(lbl1[i] == "PD"):
        label += [0]
    if(lbl1[i] == "JW"):
        label += [1]
    if(lbl1[i] == "PP"):
        label += [2]
    if(lbl1[i] == "SC"):
        label += [3]
    if(lbl1[i] == "BA"):
        label += [4]
print(label)

[0, 0, 0, 3, 1, 3, 3, 3, 3, 1, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 1, 0, 3, 4, 3, 4, 0, 0, 3, 2, 0, 0, 0, 0, 0, 4, 3, 3, 4, 4, 1, 1, 3, 0, 4, 1, 3, 3, 3, 3, 3, 0, 0, 4, 3, 0, 0, 3, 3, 3, 3, 3, 1, 0, 0, 3, 1, 0, 0, 4, 4, 3, 3, 3, 4, 2, 2, 4, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 0, 3, 3, 3, 3, 3, 1, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 0, 0, 3, 0, 3, 0, 4, 1, 0, 3, 1, 3, 3, 1, 3, 3, 3, 3, 3, 0, 0, 3, 3, 0, 3, 1, 3, 3, 3, 3, 3, 3, 0, 1, 0, 3, 3, 1, 3, 3, 3, 3, 3, 3, 0, 2, 3, 3, 3, 3, 0, 0, 0, 0, 1, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 0, 4, 4, 3, 0, 4, 0, 3, 3, 3, 3, 0, 3, 0, 3, 2, 3, 0, 3, 1, 4, 3, 0, 0, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 4, 1, 0, 0, 2, 3, 3, 3, 0, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 0, 4, 4, 4, 0, 0, 0, 0, 3, 3, 3, 3, 4, 3, 3, 4, 4, 3, 3, 1, 3, 4, 1, 3, 3, 3, 3, 4, 0, 0]


In [12]:
label = np.array(label)
print(label.shape)

(291,)


In [16]:
data2 = data1/data
print(data2.shape)

(291, 21)


In [17]:
print(data2)
print(data2.max())
data2 = data2/data2.max()
print(data2.max())

[[4.44215485 4.03107831 4.26765747 ... 2.40375025 2.46242098 4.58025963]
 [7.60654049 5.70766584 2.76754234 ... 2.65462117 5.10168325 5.85614705]
 [4.96507085 6.44693126 2.97953808 ... 1.3642738  2.67007095 5.48129715]
 ...
 [7.51062586 5.408194   7.23937542 ... 3.3320972  3.04228477 8.16708078]
 [2.62114386 3.38603388 4.27185247 ... 9.11640627 8.9000907  3.21773101]
 [3.44106686 2.14646849 3.97819856 ... 3.76544599 3.1413722  3.63696308]]
29.889381818954696
1.0


In [48]:
# 80% train 20% test (NDVIclean)
X_train = data2[0:232,:]
X_test = data2[232:,:]
y_train = label[0:232]
y_test = label[232:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(232, 21)
(59, 21)
(232,)
(59,)


In [49]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(232, 5)
(59, 5)


In [50]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 17
num_batch = 13

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 0.54471869
Epoch: 2 cost = 0.52961691
Epoch: 3 cost = 0.51456671
Epoch: 4 cost = 0.49971370
Epoch: 5 cost = 0.48528464
Epoch: 6 cost = 0.47155685
Epoch: 7 cost = 0.45881537
Epoch: 8 cost = 0.44730590
Epoch: 9 cost = 0.43719462
Epoch: 10 cost = 0.42854503
Epoch: 11 cost = 0.42131783
Epoch: 12 cost = 0.41539112
Epoch: 13 cost = 0.41059243
Epoch: 14 cost = 0.40673164
Epoch: 15 cost = 0.40362567
Epoch: 16 cost = 0.40111226
Epoch: 17 cost = 0.39905429
Epoch: 18 cost = 0.39733843
Epoch: 19 cost = 0.39587256
Epoch: 20 cost = 0.39458323
Epoch: 21 cost = 0.39341349
Epoch: 22 cost = 0.39232104
Epoch: 23 cost = 0.39127601
Epoch: 24 cost = 0.39025886
Epoch: 25 cost = 0.38925785
Epoch: 26 cost = 0.38826726
Epoch: 27 cost = 0.38728466
Epoch: 28 cost = 0.38630849
Epoch: 29 cost = 0.38533479
Epoch: 30 cost = 0.38435474
Epoch: 31 cost = 0.38335459
Epoch: 32 cost = 0.38231703
Epoch: 33 cost = 0.38122458
Epoch: 34 cost = 0.38006168
Epoch: 35 cost = 0.37881600
Epoch: 36 cost = 0.37747735
E

Epoch: 350 cost = 0.02825761
Epoch: 351 cost = 0.02807296
Epoch: 352 cost = 0.02792156
Epoch: 353 cost = 0.02776631
Epoch: 354 cost = 0.02774371
Epoch: 355 cost = 0.02768840
Epoch: 356 cost = 0.02738474
Epoch: 357 cost = 0.02723591
Epoch: 358 cost = 0.02710953
Epoch: 359 cost = 0.02698384
Epoch: 360 cost = 0.02680436
Epoch: 361 cost = 0.02661732
Epoch: 362 cost = 0.02654571
Epoch: 363 cost = 0.02649835
Epoch: 364 cost = 0.02628676
Epoch: 365 cost = 0.02609743
Epoch: 366 cost = 0.02589414
Epoch: 367 cost = 0.02579439
Epoch: 368 cost = 0.02573066
Epoch: 369 cost = 0.02557111
Epoch: 370 cost = 0.02538570
Epoch: 371 cost = 0.02520227
Epoch: 372 cost = 0.02496908
Epoch: 373 cost = 0.02486618
Epoch: 374 cost = 0.02478695
Epoch: 375 cost = 0.02455361
Epoch: 376 cost = 0.02438778
Epoch: 377 cost = 0.02423071
Epoch: 378 cost = 0.02428856
Epoch: 379 cost = 0.02410442
Epoch: 380 cost = 0.02385354
Epoch: 381 cost = 0.02364536
Epoch: 382 cost = 0.02349578
Epoch: 383 cost = 0.02337717
Epoch: 384 cos

Epoch: 679 cost = 0.00077054
Epoch: 680 cost = 0.00813248
Epoch: 681 cost = 0.00557241
Epoch: 682 cost = 0.00056202
Epoch: 683 cost = 0.00055713
Epoch: 684 cost = 0.00047023
Epoch: 685 cost = 0.00050090
Epoch: 686 cost = 0.00043915
Epoch: 687 cost = 0.00038685
Epoch: 688 cost = 0.00034426
Epoch: 689 cost = 0.00037376
Epoch: 690 cost = 0.00039738
Epoch: 691 cost = 0.00035834
Epoch: 692 cost = 0.00037523
Epoch: 693 cost = 0.00045125
Epoch: 694 cost = 0.00042242
Epoch: 695 cost = 0.00034940
Epoch: 696 cost = 0.00041038
Epoch: 697 cost = 0.00046807
Epoch: 698 cost = 0.00043562
Epoch: 699 cost = 0.00037574
Epoch: 700 cost = 0.00038427
Epoch: 701 cost = 0.00038527
Epoch: 702 cost = 0.00041877
Epoch: 703 cost = 0.00049937
Epoch: 704 cost = 0.00048494
Epoch: 705 cost = 0.00040317
Epoch: 706 cost = 0.00048274
Epoch: 707 cost = 0.00048106
Epoch: 708 cost = 0.00045137
Epoch: 709 cost = 0.00038204
Epoch: 710 cost = 0.00040876
Epoch: 711 cost = 0.00054898
Epoch: 712 cost = 0.00067254
Epoch: 713 cos

In [51]:
# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(label_pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [41]:
print(y_test.shape)

(59, 5)


In [42]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.5762712


In [27]:
# 70% train 30% test (NDVIclean)
# Train - 204
# Test - 87
X_train = data2[0:204,:]
X_test = data2[204:,:]
y_train = label[0:204]
y_test = label[204:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(204, 21)
(87, 21)
(204,)
(87,)


In [28]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(204, 5)
(87, 5)


In [32]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 17
num_batch = 12

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 0.51378516
Epoch: 2 cost = 0.50244562
Epoch: 3 cost = 0.49402952
Epoch: 4 cost = 0.48790626
Epoch: 5 cost = 0.48343903
Epoch: 6 cost = 0.48007733
Epoch: 7 cost = 0.47740022
Epoch: 8 cost = 0.47511788
Epoch: 9 cost = 0.47304845
Epoch: 10 cost = 0.47108728
Epoch: 11 cost = 0.46917836
Epoch: 12 cost = 0.46729255
Epoch: 13 cost = 0.46541262
Epoch: 14 cost = 0.46352470
Epoch: 15 cost = 0.46161272
Epoch: 16 cost = 0.45965624
Epoch: 17 cost = 0.45763055
Epoch: 18 cost = 0.45550797
Epoch: 19 cost = 0.45326054
Epoch: 20 cost = 0.45086245
Epoch: 21 cost = 0.44829269
Epoch: 22 cost = 0.44553574
Epoch: 23 cost = 0.44258207
Epoch: 24 cost = 0.43942698
Epoch: 25 cost = 0.43606871
Epoch: 26 cost = 0.43250571
Epoch: 27 cost = 0.42873367
Epoch: 28 cost = 0.42474212
Epoch: 29 cost = 0.42051244
Epoch: 30 cost = 0.41601835
Epoch: 31 cost = 0.41123587
Epoch: 32 cost = 0.40617152
Epoch: 33 cost = 0.40091004
Epoch: 34 cost = 0.39563167
Epoch: 35 cost = 0.39051710
Epoch: 36 cost = 0.38561380
E

Epoch: 341 cost = 0.02747012
Epoch: 342 cost = 0.02730337
Epoch: 343 cost = 0.02717002
Epoch: 344 cost = 0.02698884
Epoch: 345 cost = 0.02691688
Epoch: 346 cost = 0.02686024
Epoch: 347 cost = 0.02655973
Epoch: 348 cost = 0.02637783
Epoch: 349 cost = 0.02615527
Epoch: 350 cost = 0.02595616
Epoch: 351 cost = 0.02575756
Epoch: 352 cost = 0.02561275
Epoch: 353 cost = 0.02550985
Epoch: 354 cost = 0.02527515
Epoch: 355 cost = 0.02515932
Epoch: 356 cost = 0.02503377
Epoch: 357 cost = 0.02472523
Epoch: 358 cost = 0.02448509
Epoch: 359 cost = 0.02465812
Epoch: 360 cost = 0.02446017
Epoch: 361 cost = 0.02421443
Epoch: 362 cost = 0.02402736
Epoch: 363 cost = 0.02374634
Epoch: 364 cost = 0.02359173
Epoch: 365 cost = 0.02335072
Epoch: 366 cost = 0.02323648
Epoch: 367 cost = 0.02306824
Epoch: 368 cost = 0.02289196
Epoch: 369 cost = 0.02273016
Epoch: 370 cost = 0.02271997
Epoch: 371 cost = 0.02253122
Epoch: 372 cost = 0.02220904
Epoch: 373 cost = 0.02196597
Epoch: 374 cost = 0.02178848
Epoch: 375 cos

Epoch: 649 cost = 0.00041600
Epoch: 650 cost = 0.00039134
Epoch: 651 cost = 0.00034276
Epoch: 652 cost = 0.00037916
Epoch: 653 cost = 0.00042793
Epoch: 654 cost = 0.00038802
Epoch: 655 cost = 0.00038032
Epoch: 656 cost = 0.00036791
Epoch: 657 cost = 0.00035669
Epoch: 658 cost = 0.00034147
Epoch: 659 cost = 0.00038507
Epoch: 660 cost = 0.00041737
Epoch: 661 cost = 0.00034016
Epoch: 662 cost = 0.00041537
Epoch: 663 cost = 0.00039344
Epoch: 664 cost = 0.00033330
Epoch: 665 cost = 0.00038905
Epoch: 666 cost = 0.00039205
Epoch: 667 cost = 0.00053851
Epoch: 668 cost = 0.00061362
Epoch: 669 cost = 0.00039957
Epoch: 670 cost = 0.00037884
Epoch: 671 cost = 0.00047375
Epoch: 672 cost = 0.00061413
Epoch: 673 cost = 0.00346419
Epoch: 674 cost = 0.00231530
Epoch: 675 cost = 0.00044289
Epoch: 676 cost = 0.00036558
Epoch: 677 cost = 0.00029804
Epoch: 678 cost = 0.00031352
Epoch: 679 cost = 0.00035750
Epoch: 680 cost = 0.00029525
Epoch: 681 cost = 0.00029067
Epoch: 682 cost = 0.00029335
Epoch: 683 cos

Epoch: 947 cost = 0.00023450
Epoch: 948 cost = 0.00020735
Epoch: 949 cost = 0.00022171
Epoch: 950 cost = 0.00026774
Epoch: 951 cost = 0.00021165
Epoch: 952 cost = 0.00020937
Epoch: 953 cost = 0.00027824
Epoch: 954 cost = 0.00026536
Epoch: 955 cost = 0.00020376
Epoch: 956 cost = 0.00021525
Epoch: 957 cost = 0.00028798
Epoch: 958 cost = 0.00016633
Epoch: 959 cost = 0.00026742
Epoch: 960 cost = 0.00024501
Epoch: 961 cost = 0.00022148
Epoch: 962 cost = 0.00036842
Epoch: 963 cost = 0.00025790
Epoch: 964 cost = 0.00026115
Epoch: 965 cost = 0.00022107
Epoch: 966 cost = 0.00025244
Epoch: 967 cost = 0.00018127
Epoch: 968 cost = 0.00018083
Epoch: 969 cost = 0.00021581
Epoch: 970 cost = 0.00020164
Epoch: 971 cost = 0.00031744
Epoch: 972 cost = 0.00019892
Epoch: 973 cost = 0.00016125
Epoch: 974 cost = 0.00026794
Epoch: 975 cost = 0.00024917
Epoch: 976 cost = 0.00034666
Epoch: 977 cost = 0.00024933
Epoch: 978 cost = 0.00026356
Epoch: 979 cost = 0.00020640
Epoch: 980 cost = 0.00031181
Epoch: 981 cos

In [34]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.57471263
